<img style="float: left;" src="D2Klogo.png" width="20%"> 

<center>

# Welcome to DSCI 101 #
### Introduction to Data Science ###

### Code Demo for Week 5 ###

In [ ]:
# import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
# ignores warning message
warnings.filterwarnings('ignore')

# Joining DataFrames #

Often data is spread across multiple tables. Joining provides us with a way to naturally combine related tables and create a richer dataset with more variables/columns.

Let's start by reading data from the csv files: `elections.csv` and `presidents.csv`.

In [ ]:
elections = pd.read_csv("elections.csv")
elections.head()

In [ ]:
presidents = pd.read_csv("presidents.csv")
presidents.head()

To join tables `df1` and `df2`, we call the function `df1.merge(df2)`. Merge is just the word that the authors of pandas picked for joining tables.

Here is the documentation:[`df.merge`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html#pandas.DataFrame.merge)

Note: Unfortunately, Pandas also has a function called `df.join`. This is a limited version of `merge`, which basically only allow you to merge tables with matched index. For ths sake of generality, we will only use `merge` in this class.

I can use the merge function to combine these two tables:

In [ ]:
# pay attention to the syntax df1.merge(df2, ...)
# how: inner, outer, left or right join
# left_on: column name in df1
# right_on: column name in df2
# match up left_on with right_on and combine columns from df1 and df2
elections.merge(presidents, 
            how = "inner",
            left_on = "Candidate", right_on = "President")

# Inner Join

The normal join only keeps rows that match the join predicate.

<img src="inner_join.png" alt="Inner Join" width="150px">

In [ ]:
joined = elections.merge(presidents, 
            how = "inner",
            left_on = "Candidate", right_on = "President")
joined.head()

Notice that:
1. The output dataframe only contains rows that have names in both tables.  For example, presidents before 1824 do not appear in the joined table because there was no popular vote before 1824.
1. The name `Andrew Jackson` occurred three times in the election table and shows up three times in the output. 
1. `Grover Cleveland` occurs six times! Twice for every election he was in. This is because he appears three times in the `elections` table and twice in the `presidents` table. This results in 3 x 2 = 6 pairs.
1. Several presidents are missing because their names are not an exact match. For example, John F. Kennedy is "John Kennedy" in the elections table and "John F. Kennedy" in the presidents table.

# Right Join

The above join was an inner join.  What if we wanted to keep all of the presidents and leave missing data for years when there was no popular vote? In this case we'd do a "right" join, where we make sure to include EVERY row from our right dataframe, in this case `presidents`.

<img src="right_join.png" alt="Right Join" width="150px">

In [ ]:
right_joined = elections.merge(presidents, how = "right", left_on = "Candidate", right_on = "President")
right_joined

# Left Join

Same as inner join but if there is a row on the left table that failed to join with a row on the right table it is also included with null values in place of all entries in the right table.

So if we also want to include candidates not in the presidents table (e.g. because they had never won), we can use a "left" join.

<img src="left_join.png" alt="Left Join" width="150px">

In [ ]:
left_joined = elections.merge(presidents, how = "left", left_on = "Candidate", right_on = "President")
left_joined

# Outer Join

If we wanted to keep both, we can instead do an "outer join". The outer ensures that at least one row from both tables appears in the final result even if there are no matches by adding null values. 


<img src="full_join.png" alt="Full Join" width="150px">

In [ ]:
outer_joined = elections.merge(presidents, how = "outer", left_on = "Candidate", right_on = "President")
outer_joined

## Combining `merge` and Other Methods ##

### Check missing values ###

Some useful df methods for checking, dropping and filling in missing values.
- [`series.isna()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.isna.html)
- [`df.isna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isna.html)
- [`df.dropna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html)
- [`df.fillna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html)

**Practice**:
After reading the documentations of these methods, try the following by yourself using the `outer_joined` as your df:
- How to find the number of missing values in each column quickly?
- How to find rows corresponding to missing values in a specific column, say `President`?
- How to drop rows corresponding to missing values in some specific columns? Let's drop all the rows if there are missing values in the following columns:  
    - 'Year','Candidate','Party','Popular vote','Result','%'
- After you drop those rows, you want to check: are there any rows that the column `President` is missing, but the `Result` is win? There should not be! But how to find those rows if there is any?

In [ ]:
# summarize number of missing values for each column
...

In [ ]:
# how to find missing records for certain column
...

In [ ]:
# drop rows for missing values in certain columns
...
# check indeed they are dropped
...

In [ ]:
# if the result is loss, they should be missing, 
# but winners, why are they missing?
...

In [ ]:
# what is going on here? 
# perhaps check the original df on presidents again
# and spot check a few examps from above, say Barack Obama
# what is your conclusion and how can we fix this issue?
...

Sometime manual intervention is inevitable when dealing with messy real-world data. You will be able to practice this in your lab assignment!

### Ages of Presidents ###

Merged DataFrames are handy because for example we can use them to plot, e.g. the age of each president when they were elected.

In [ ]:
# this is the inner join version
# no missing values
# let's we only look at winners
winners = joined[joined['Result']=='win']
winners.head()

In order to calculate age, we need the birth year seperated from the `Date of birth` column, which is a string. Check out the string method:
[`str.split`](https://docs.python.org/3/library/stdtypes.html?highlight=str%20split#str.split)

In [ ]:
# pat= where you want to split the string
# expand=True returns splitted string as a df, otherwise returns series
birthday = winners["Date of birth"].str.split(pat=",", expand=True)
birthday

In [ ]:
# splited string is still string
# but we need numbers to do calculation
# cast data type with df[].astype
birth_year = birthday[1].astype(int)
birth_year

Another way to get the year is to convert the column `Date of birth` from a string into a correct datetime format using [`pandas.to_datetime`](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html), then it is easy to extrat year, month and day from a datetime using [`series.dt.year`](https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.year.html)

In [ ]:
winners["DOB_datetime"] = pd.to_datetime(winners["Date of birth"])
winners.head()

In [ ]:
winners["DOB_year"] = winners["DOB_datetime"].dt.year
winners.head()

In [ ]:
# calculate the age and add to df as a column named "Age"
# I will get a warning but you can ignore it
winners['Age'] = winners['Year'] - winners["DOB_year"]
winners.head()

In [ ]:
# make a plot to show the trend of US presidents' age 
winners.plot(kind="scatter", x="Year", y="Age", title="Age of Presidents");

**Practice**:

Let's use a toy example to practice joining dataframes and pulling information together. First run the following cells to creat the two dataframes:

In [ ]:
# a dataframe of drinks
drinks = pd.DataFrame([
    ['Green Tea', 'Tea One', 4],
    ['Boba Tea', 'Tea One', 5],
    ['Espresso', 'Nefeli',  2],
    ['Latte',    'Nefeli',  3],
    ['Espresso', "Abe's",   2],
    ['Macchiato', "Abe's",  3]],
    index=[1,2,3,4,5,6],
    columns=['Drink', 'Cafe', 'Price']
)
drinks

In [ ]:
# a data frame of cafes and the discount they are running
discounts = pd.DataFrame(
    {'Coupon % off': np.array([15, 10, 5]),
    'Location':np.array(['Tea One', 'Nefeli', "Abe's"])},
    index = [1, 2, 3]
)
discounts

**Question:**
How to get discounted price for each drink? Of course in this case you can calculate each one manually, but think of these tables as if they have thousands or millions of rows so you want a way to automate the calculation by code!

**Hint**: First create a dataframe by joining drinks and discounts (what would be the common column to merge on?). Then create a new column in your merged data frame to calculate the discounted price.

In [ ]:
### fill in your code here
...